In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
import spacy
from spacy_langdetect import LanguageDetector
from spacy.language import Language
nlp = spacy.load('en_core_web_sm')


def create_lang_detector(nlp, name):
    return LanguageDetector()

Language.factory("language_detector", func=create_lang_detector)

nlp.add_pipe('language_detector', last=True)

In [3]:
df = pd.read_csv("train_news.csv", index_col=0)
df

id                                           headline  \
0       9653  Ethics Questions Dogged Agriculture Nominee as...   
1      10041  U.S. Must Dig Deep to Stop Argentina’s Lionel ...   
2      19113  Cotton to House: ’Do Not Walk the Plank and Vo...   
3       6868  Paul LePage, Besieged Maine Governor, Sends Co...   
4       7596                       A Digital 9/11 If Trump Wins   
...      ...                                                ...   
20795   5671                                                NaN   
20796  14831        Albert Pike and the European Migrant Crisis   
20797  18142  Dakota Access Caught Infiltrating Protests to ...   
20798  12139  How to Stretch the Summer Solstice - The New Y...   
20799  15660  Emory University to Pay for ’100 Percent’ of U...   

                       written_by  \
0      Eric Lipton and Steve Eder   
1                 David Waldstein   
2                         Pam Key   
3                    Jess Bidgood   
4               Finian Cunningham   
...                           ...   
20795              NeverSurrender   
20796               Rixon Stewart   
20797                 Eddy Lavine   
20798              Alison S. Cohn   
20799                Tom Ciccotta   

                                                    news  label  
0      WASHINGTON  —   In Sonny Perdue’s telling, Geo...      0  
1      HOUSTON  —   Venezuela had a plan. It was a ta...      0  
2      Sunday on ABC’s “This Week,” while discussing ...      0  
3      AUGUSTA, Me.  —   The beleaguered Republican g...      0  
4        Finian Cunningham has written extensively on...      1  
...                                                  ...    ...  
20795  No, you'll be a dog licking of the vomit of yo...      1  
20796  By Rixon Stewart on November 5, 2016 Rixon Ste...      1  
20797  posted by Eddie You know the Dakota Access Pip...      1  
20798  It’s officially summer, and the Society Boutiq...      0  
20799  Emory University in Atlanta, Georgia, has anno...      0  

[20800 rows x 5 columns]

In [4]:
df.isnull().sum()

id               0
headline       558
written_by    1957
news            39
label            0
dtype: int64

In [5]:
df['Headline_News'] = df['news'].fillna('') +' '+ df['headline'].fillna('')
df

id                                           headline  \
0       9653  Ethics Questions Dogged Agriculture Nominee as...   
1      10041  U.S. Must Dig Deep to Stop Argentina’s Lionel ...   
2      19113  Cotton to House: ’Do Not Walk the Plank and Vo...   
3       6868  Paul LePage, Besieged Maine Governor, Sends Co...   
4       7596                       A Digital 9/11 If Trump Wins   
...      ...                                                ...   
20795   5671                                                NaN   
20796  14831        Albert Pike and the European Migrant Crisis   
20797  18142  Dakota Access Caught Infiltrating Protests to ...   
20798  12139  How to Stretch the Summer Solstice - The New Y...   
20799  15660  Emory University to Pay for ’100 Percent’ of U...   

                       written_by  \
0      Eric Lipton and Steve Eder   
1                 David Waldstein   
2                         Pam Key   
3                    Jess Bidgood   
4               Finian Cunningham   
...                           ...   
20795              NeverSurrender   
20796               Rixon Stewart   
20797                 Eddy Lavine   
20798              Alison S. Cohn   
20799                Tom Ciccotta   

                                                    news  label  \
0      WASHINGTON  —   In Sonny Perdue’s telling, Geo...      0   
1      HOUSTON  —   Venezuela had a plan. It was a ta...      0   
2      Sunday on ABC’s “This Week,” while discussing ...      0   
3      AUGUSTA, Me.  —   The beleaguered Republican g...      0   
4        Finian Cunningham has written extensively on...      1   
...                                                  ...    ...   
20795  No, you'll be a dog licking of the vomit of yo...      1   
20796  By Rixon Stewart on November 5, 2016 Rixon Ste...      1   
20797  posted by Eddie You know the Dakota Access Pip...      1   
20798  It’s officially summer, and the Society Boutiq...      0   
20799  Emory University in Atlanta, Georgia, has anno...      0   

                                           Headline_News  
0      WASHINGTON  —   In Sonny Perdue’s telling, Geo...  
1      HOUSTON  —   Venezuela had a plan. It was a ta...  
2      Sunday on ABC’s “This Week,” while discussing ...  
3      AUGUSTA, Me.  —   The beleaguered Republican g...  
4        Finian Cunningham has written extensively on...  
...                                                  ...  
20795  No, you'll be a dog licking of the vomit of yo...  
20796  By Rixon Stewart on November 5, 2016 Rixon Ste...  
20797  posted by Eddie You know the Dakota Access Pip...  
20798  It’s officially summer, and the Society Boutiq...  
20799  Emory University in Atlanta, Georgia, has anno...  

[20800 rows x 6 columns]

In [6]:
df.isnull().sum()

id                  0
headline          558
written_by       1957
news               39
label               0
Headline_News       0
dtype: int64

In [8]:
df[df['headline'].isnull()]['label'].value_counts()

1    558
Name: label, dtype: int64

### 'news'

In [9]:
df[df['news'].isnull()]['label'].value_counts()

1    39
Name: label, dtype: int64

In [10]:
df[df['news'].isnull()]

id                                           headline written_by  \
556     9454  Il saoule tout le monde avec son analyse polit...        NaN   
1642   11486  Pour booster les ventes, Dassault offre un por...        NaN   
1765     573  Le top des recherches Google passe en top des ...        NaN   
1968    9446  Trop essoufflé après avoir cherché ses affaire...        NaN   
2200    3729  Les Américains ne sont plus qu’à quelques heur...        NaN   
3183   13107  Les gardes-frontières se mettent en alerte pou...        NaN   
3927    4358  Ne supportant plus l’ambiance de la campagne é...        NaN   
4333   14499  Primaire – François Fillon se désiste au profi...        NaN   
4746    2148  Gorafi Magazine: Barack Obama « Je vous ai déj...        NaN   
4747    8649  Donald Trump s’excuse pour toutes les minorité...        NaN   
4942   10867    Live Soirée présidentielle US 2016 >> Le Gorafi        NaN   
6849    6215  New-York – Le lâcher de confettis prévu à Time...        NaN   
7100    3329  GuinessBook : 100 millions d’Américains batten...        NaN   
7271   19157  110% des Américains assurent qu’ils continuero...        NaN   
7388   11450  Vladimir Poutine est élu 45e Président des Eta...        NaN   
7727   18757               Donald Trump en 5 dates >> Le Gorafi        NaN   
7951    2169  #NDDL: Pour fêter l’autorisation des travaux, ...        NaN   
8535   12835  [Vidéo] Découvrez le Gorafi de l’Année 2017 >>...        NaN   
9719    9524  Elections US : les premières estimations donne...        NaN   
10690  16126  Vendée Globe – Un skipper fait demi-tour après...        NaN   
11784  14933  La sortie du dernier livre inédit du Gorafi éc...        NaN   
11850  19388  Heureux et souriant, il a la chance de toujour...        NaN   
11937   4288            Hillary Clinton en 5 dates >> Le Gorafi        NaN   
12315  10466  Sur Twitter, Donald Trump se félicite d’avoir ...        NaN   
13452    142  Gorafi Magazine : Entretien exclusif avec Bara...        NaN   
14178  13020         Horoscope du 21 novembre 2016 >> Le Gorafi        NaN   
15235   1911  Grand-Prix du Brésil – Romain Grosjean obtient...        NaN   
16078   2793  «Pourquoi je suis candidat à la présidentielle...        NaN   
16690   8922  La Trump Tower cambriolée – les voleurs seraie...        NaN   
16863   1200  La Corée du Nord annonce avoir envoyé un missi...        NaN   
16884  13915  Elle met en fuite un braqueur en lui jetant de...        NaN   
16901  18479  Il se fait cryogéniser dans l’espoir de voir u...        NaN   
18470  19227  Des millions d’Américains recherchent massivem...        NaN   
18512  12056  Après le succès de « Mariés au premier regard ...        NaN   
18683  12460  La voyante lisait l’avenir dans les cartes ble...        NaN   
19436   6680  Débat Primaire de la Droite : les Français pas...        NaN   
19507   8908  “Notre batal a coulé” par l’homme qui fait des...        NaN   
20512   9350  La Russie a profité de l’élection américaine p...        NaN   
20603   5717  SPA : Enfermé depuis trois jours dans une cage...        NaN   

      news  label                                      Headline_News  
556    NaN      1   Il saoule tout le monde avec son analyse poli...  
1642   NaN      1   Pour booster les ventes, Dassault offre un po...  
1765   NaN      1   Le top des recherches Google passe en top des...  
1968   NaN      1   Trop essoufflé après avoir cherché ses affair...  
2200   NaN      1   Les Américains ne sont plus qu’à quelques heu...  
3183   NaN      1   Les gardes-frontières se mettent en alerte po...  
3927   NaN      1   Ne supportant plus l’ambiance de la campagne ...  
4333   NaN      1   Primaire – François Fillon se désiste au prof...  
4746   NaN      1   Gorafi Magazine: Barack Obama « Je vous ai dé...  
4747   NaN      1   Donald Trump s’excuse pour toutes les minorit...  
4942   NaN      1    Live Soirée présidentielle US 2016 >> Le Gorafi  
6849   NaN      1   New-York – Le lâc

### 'written_by'

In [12]:
len(df['written_by'].unique())

4202

In [7]:
df[df['written_by'].isnull()]['label'].value_counts()

1    1931
0      26
Name: label, dtype: int64

In [6]:
df[df['written_by'].isnull()][df[df['written_by'].isnull()]['label']==0]

id                                           headline written_by  \
244    13993  Full Transcript and Video: Trump Picks Neil Go...        NaN   
1894   17993  California Today: Airbnb, a Hometown Success (...        NaN   
1954   15876  9 New Books We Recommend This Week - The New Y...        NaN   
2583    5911  Keep Your Family Close and Your Roomba Closer ...        NaN   
2703   14228  Transcript: President Obama on What Books Mean...        NaN   
2798   15893  Reaction to a Supreme Court Tie on Immigration...        NaN   
2863    7032  Reactions to the Supreme Court Ruling on Texas...        NaN   
3041   12729  The Best and Worst of the Golden Globes - The ...        NaN   
5272   10204  Full Transcript and Video: Trump News Conferen...        NaN   
5474    3675  2 TV Shows, 4 Books and 7 Viral Videos for You...        NaN   
7354     685  The New York Times to Offer Open Access on Web...        NaN   
8370   19265                     My Canada - The New York Times        NaN   
9328    2785    How the Poll Was Conducted - The New York Times        NaN   
10772   8397    Fran Lebowitz: By the Book - The New York Times        NaN   
11198  17837  At the Tonys, Moments to Remember - The New Yo...        NaN   
11636  13995  To Our Readers, From the Publisher and Executi...        NaN   
12088   5608  As Hurricane Matthew Nears U.S., Residents Get...        NaN   
12570      8  Excerpts From a Draft Script for Donald Trump’...        NaN   
12669   3470  ‘My First Gay Bar’: Rachel Maddow, Andy Cohen ...        NaN   
12781  17943  Hurricane Matthew Avoids Direct Hit on Florida...        NaN   
13306  13267  Notes From Aleppo: Intimate Glimpses of War-Ra...        NaN   
16861  19704     100 Notable Books of 2016 - The New York Times        NaN   
18914  16403           Grammy Winners 2017 - The New York Times        NaN   
19800   3203  37 Who Saw Murder Didn’t Call the Police - The...        NaN   
20304   5239  Text of F.B.I. Director’s Remarks on Investiga...        NaN   
20315  11986  Hurricane Matthew Approaches Florida Governor ...        NaN   

                                                    news  label  \
244    President Trump announced on Tuesday that Neil...      0   
1894   Good morning. This is the last day of our test...      0   
1954   I know, the number of big books out each fall ...      0   
2583   How lovable can a robot be? A study of Roomba ...      0   
2703   Michiko Kakutani, the chief book critic for Th...      0   
2798   The Supreme Court’s   vote in a major immigrat...      0   
2863   In a major abortion ruling Monday, the Supreme...      0   
3041   Here’s a look at the most memorable moments fr...      0   
5272   President Trump on Thursday announced his new ...      0   
5474   Summer’s almost over. Here’s a cultural diet f...      0   
7354   The New York Times is inviting readers to take...      0   
8370   Canada, our No. 1 pick for this year’s 52 Plac...      0   
9328   The latest New York   News Poll is based on te...      0   
10772  The humorist and social commentator says her i...      0   
11198  In a Tony Awards night shadowed by the tragedy...      0   
11636  When the biggest political story of the year r...      0   
12088  • Hurricane Matthew, a deadly storm with maxim...      0   
12570  Donald J. Trump is scheduled to make a highly ...      0   
12669  For generations of gays and lesbians, especial...      0   
12781  Hurricane Matthew churned north along the coas...      0   
13306  Declan Walsh, our bureau chief in Cairo, is in...      0   
16861  The year’s notable fiction, poetry and nonfict...      0   
18914  The complete list of winners: Record of the Ye...      0   
19800  For more than half an hour 38 respectable, law...      0   
20304  Following is the prepared text of remarks by D...      0   
20315  Read Friday’s Storm Watch for the latest updat...      0   

                                           Headline_News  
244    President Trump announced o

In [10]:
dict(df[df['label']==1]['written_by'].value_counts())

{'admin': 193,
 'Pakalert': 86,
 'Eddy Lavine': 85,
 'Starkman': 84,
 'Alex Ansary': 82,
 'Gillian': 82,
 'Editor': 81,
 'noreply@blogger.com (Alexander Light)': 80,
 'Anonymous': 77,
 'Dave Hodges': 77,
 'IWB': 75,
 'BareNakedIslam': 74,
 'The European Union Times': 74,
 'Activist Post': 72,
 'The Doc': 69,
 'EdJenner': 69,
 'Henry Wolff': 67,
 'Mac Slavo': 66,
 'Iron Sheik': 66,
 'Kaitlyn Stegall': 65,
 'noreply@blogger.com (Der Postillon)': 64,
 'Jason Ditz': 64,
 'Heather Callaghan': 62,
 'David Stockman': 61,
 'Geoffrey Grider': 61,
 'King World News': 60,
 'shorty': 58,
 'Consortiumnews.com': 58,
 'The Saker': 55,
 '-NO AUTHOR-': 54,
 'Dikran Arakelian (noreply@blogger.com)': 51,
 'wmw_admin': 51,
 'Amy Moreno': 48,
 'Daniel Greenfield': 47,
 'Andrew Anglin': 47,
 'stevew': 45,
 'Jafe Arnoldski (noreply@blogger.com)': 44,
 'Howard Portnoy': 44,
 'pcr3': 42,
 'Arnold Monteverde (noreply@blogger.com)': 41,
 'Admin': 41,
 'beforeitsnews.com': 40,
 'Guest': 39,
 'Mike Rivero': 39,
 '

In [11]:
search_values = [' il ',' un ', ' contro ', ' la ', ' è ', ' una ', ' Buena ', ' de ', ' equipo '] # Common Spanish words
a = df[df.Headline_News.str.contains('|'.join(search_values ),case=False)] # Extracting rows which contain spanish reviews

# for i in a.Headline_News:
#     print(i.replace('\n',"  "))
#     print("-----------------------------")
# print(len(a))
# print(a.index)
a

id                                           headline  \
1      10041  U.S. Must Dig Deep to Stop Argentina’s Lionel ...   
13     10200  Donald Trump Is The Candidate For Americans, N...   
19     19716  Cities Vow to Fight Trump on Immigration, Even...   
29      4934  Review: Bruno Mars Delivers Decades of Funk in...   
45      2095  6-minute analysis: Will FBI agents break ‘Empe...   
...      ...                                                ...   
20761  12613  Graft Allegations Grow Against François Fillon...   
20766    491   ‘They Will Have to Die Now’ - The New York Times   
20769  17683  #BoycottComedian…ROBERT DENIRO Wanted “To Punc...   
20784  10501  Crash Near Finish Throws Tour de France Into C...   
20786   9792  The Lost Footage of Marilyn Monroe - The New Y...   

                             written_by  \
1                       David Waldstein   
13                       Luke Stranahan   
19     Jennifer Medina and Jess Bidgood   
29                       Jon Caramanica   
45                          Carl Herman   
...                                 ...   
20761                  Aurelien Breeden   
20766                      James Verini   
20769                          EdJenner   
20784                        Ian Austen   
20786                  Helene Stapinski   

                                                    news  label  \
1      HOUSTON  —   Venezuela had a plan. It was a ta...      0   
13       Donald Trump Is The Candidate For Americans,...      1   
19     LOS ANGELES  —   Here in Los Angeles, where ne...      0   
29     The Bruno Mars   machine has been one of pop’s...      0   
45     hat tip: What Really Happened \nAlex Jones’ 6-...      1   
...                                                  ...    ...   
20761  PARIS  —   New embezzlement allegations emerge...      0   
20766  Before dawn on a Sunday late last month, a bat...      0   
20769  Go to Article Donald Trump was willing to give...      1   
20784     France  —   His bike crushed under a motorc...      0   
20786  It happened one night in the late summer of 19...      0   

                                           Headline_News  
1      HOUSTON  —   Venezuela had a plan. It was a ta...  
13       Donald Trump Is The Candidate For Americans,...  
19     LOS ANGELES  —   Here in Los Angeles, where ne...  
29     The Bruno Mars   machine has been one of pop’s...  
45     hat tip: What Really Happened \nAlex Jones’ 6-...  
...                                                  ...  
20761  PARIS  —   New embezzlement allegations emerge...  
20766  Before dawn on a Sunday late last month, a bat...  
20769  Go to Article Donald Trump was willing to give...  
20784     France  —   His bike crushed under a motorc...  
20786  It happened one night in the late summer of 19...  

[1878 rows x 6 columns]

In [19]:
def lang_detect(text):
    doc = nlp(text)
    return doc._.language['language']

In [44]:
df_not_null_news = df[~df['news'].isnull()].reset_index() 
df_not_null_news

index     id                                           headline  \
0          0   9653  Ethics Questions Dogged Agriculture Nominee as...   
1          1  10041  U.S. Must Dig Deep to Stop Argentina’s Lionel ...   
2          2  19113  Cotton to House: ’Do Not Walk the Plank and Vo...   
3          3   6868  Paul LePage, Besieged Maine Governor, Sends Co...   
4          4   7596                       A Digital 9/11 If Trump Wins   
...      ...    ...                                                ...   
20756  20795   5671                                                NaN   
20757  20796  14831        Albert Pike and the European Migrant Crisis   
20758  20797  18142  Dakota Access Caught Infiltrating Protests to ...   
20759  20798  12139  How to Stretch the Summer Solstice - The New Y...   
20760  20799  15660  Emory University to Pay for ’100 Percent’ of U...   

                       written_by  \
0      Eric Lipton and Steve Eder   
1                 David Waldstein   
2                         Pam Key   
3                    Jess Bidgood   
4               Finian Cunningham   
...                           ...   
20756              NeverSurrender   
20757               Rixon Stewart   
20758                 Eddy Lavine   
20759              Alison S. Cohn   
20760                Tom Ciccotta   

                                                    news  label  \
0      WASHINGTON  —   In Sonny Perdue’s telling, Geo...      0   
1      HOUSTON  —   Venezuela had a plan. It was a ta...      0   
2      Sunday on ABC’s “This Week,” while discussing ...      0   
3      AUGUSTA, Me.  —   The beleaguered Republican g...      0   
4        Finian Cunningham has written extensively on...      1   
...                                                  ...    ...   
20756  No, you'll be a dog licking of the vomit of yo...      1   
20757  By Rixon Stewart on November 5, 2016 Rixon Ste...      1   
20758  posted by Eddie You know the Dakota Access Pip...      1   
20759  It’s officially summer, and the Society Boutiq...      0   
20760  Emory University in Atlanta, Georgia, has anno...      0   

                                           Headline_News  
0      WASHINGTON  —   In Sonny Perdue’s telling, Geo...  
1      HOUSTON  —   Venezuela had a plan. It was a ta...  
2      Sunday on ABC’s “This Week,” while discussing ...  
3      AUGUSTA, Me.  —   The beleaguered Republican g...  
4        Finian Cunningham has written extensively on...  
...                                                  ...  
20756  No, you'll be a dog licking of the vomit of yo...  
20757  By Rixon Stewart on November 5, 2016 Rixon Ste...  
20758  posted by Eddie You know the Dakota Access Pip...  
20759  It’s officially summer, and the Society Boutiq...  
20760  Emory University in Atlanta, Georgia, has anno...  

[20761 rows x 7 columns]

In [45]:
j = 0
ind = []
for i in df_not_null_news.loc[:,'news']:
    if lang_detect(i)!='en':
        ind.append(j)
    j=j+1
len(ind)

593

In [49]:
non_eng_news = df_not_null_news.loc[ind,:]
non_eng_news

index     id                                           headline  \
46        46  13433                    مذكرة مفاهيمية عن رئاسة الروسية   
47        47  11814  Las imágenes libres de derechos más destacadas...   
79        79  17351  Thomas Frank Explores Whether Hillary Clinton ...   
81        81  12930  Rajoy anuncia el inicio de una "nueva etapa po...   
111      111  16093  حميميم: 44 عملية قصف من قبل المسلحين خلال السا...   
...      ...    ...                                                ...   
20598  20637   4488  Miss Islandia abandona concurso internacional ...   
20685  20724  19722  Birleşik Krallık, Suriye’de cihatçıları eğitme...   
20687  20726  15619  صادرات النفط الكويتي إلى الصين تسجل رقما قياسي...   
20691  20730  13262  Путин: Запад постоянно наступает на одни и те ...   
20714  20753   9516  Las imágenes libres de derechos más destacadas...   

            written_by                                               news  \
46     voltairenet.org  اجتماع لمجلس الأمن من 28 أكتوبر 2016\nمذكرة مف...   
47       Tomás Fuentes  Las imágenes libres de derechos más destacadas...   
79                 NaN                                                      
81                 NaN  (actualizada a las 21:34 26.10.2016) URL corto...   
111                NaN  انسخ الرابط http://ar.rt.com/i5hy أعلن مركز ال...   
...                ...                                                ...   
20598              NaN  Miss Islandia abandona concurso internacional ...   
20685     Osman Soysal  Birleşik Krallık, Suriye’de cihatçıları eğitme...   
20687              NaN  صادرات النفط الكويتي إلى الصين تسجل رقما قياسي...   
20691              NaN  0 комментариев 4 поделились \n"На примере терр...   
20714    Tomás Fuentes  Las imágenes libres de derechos más destacadas...   

       label                                      Headline_News  
46         1  اجتماع لمجلس الأمن من 28 أكتوبر 2016\nمذكرة مف...  
47         1  Las imágenes libres de derechos más destacadas...  
79         1    Thomas Frank Explores Whether Hillary Clinto...  
81         1  (actualizada a las 21:34 26.10.2016) URL corto...  
111        1  انسخ الرابط http://ar.rt.com/i5hy أعلن مركز ال...  
...      ...                                                ...  
20598      1  Miss Islandia abandona concurso internacional ...  
20685      1  Birleşik Krallık, Suriye’de cihatçıları eğitme...  
20687      1  صادرات النفط الكويتي إلى الصين تسجل رقما قياسي...  
20691      1  0 комментариев 4 поделились \n"На примере терр...  
20714      1  Las imágenes libres de derechos más destacadas...  

[593 rows x 7 columns]

In [50]:
non_eng_news.to_csv("Non_English_News.csv",index=False)

In [51]:
non_eng_news['label'].value_counts()

1    590
0      3
Name: label, dtype: int64

In [52]:
non_eng_news[non_eng_news['label']==0]

index     id                                           headline  \
5253    5264   8130  Regards ‘changés’ et ‘langues déliées’ : Des m...   
9137    9155  18567  Police: Chicago Torture Victim’s Parents Recei...   
13550  13575   7141  When I’m Mistakenly Put on an Email Chain, Sho...   

            written_by                                               news  \
5253   Lillie Dremeaux  Les voix des femmes musulmanes ont été quasime...   
9137    Breitbart News                                      advertisement   
13550    Daniel Victor                                                No.   

       label                                      Headline_News  
5253       0  Les voix des femmes musulmanes ont été quasime...  
9137       0  advertisement Police: Chicago Torture Victim’s...  
13550      0  No. When I’m Mistakenly Put on an Email Chain,...

In [58]:
lang_detect(non_eng_news.loc[13550,'Headline_News'])

'en'

In [59]:
j = 0
ind1 = []
for i in df.loc[:,'Headline_News']:
    if lang_detect(i)!='en':
        ind1.append(j)
    j=j+1
len(ind1)

544

In [60]:
len(ind1)

544

In [62]:
df.loc[ind1,:]['label'].value_counts()

1    543
0      1
Name: label, dtype: int64

In [63]:
df['Headline_Written_News'] = df['headline'].fillna('') + ' ' + df['written_by'].fillna('') + ' ' + df['news'].fillna('')
df

id                                           headline  \
0       9653  Ethics Questions Dogged Agriculture Nominee as...   
1      10041  U.S. Must Dig Deep to Stop Argentina’s Lionel ...   
2      19113  Cotton to House: ’Do Not Walk the Plank and Vo...   
3       6868  Paul LePage, Besieged Maine Governor, Sends Co...   
4       7596                       A Digital 9/11 If Trump Wins   
...      ...                                                ...   
20795   5671                                                NaN   
20796  14831        Albert Pike and the European Migrant Crisis   
20797  18142  Dakota Access Caught Infiltrating Protests to ...   
20798  12139  How to Stretch the Summer Solstice - The New Y...   
20799  15660  Emory University to Pay for ’100 Percent’ of U...   

                       written_by  \
0      Eric Lipton and Steve Eder   
1                 David Waldstein   
2                         Pam Key   
3                    Jess Bidgood   
4               Finian Cunningham   
...                           ...   
20795              NeverSurrender   
20796               Rixon Stewart   
20797                 Eddy Lavine   
20798              Alison S. Cohn   
20799                Tom Ciccotta   

                                                    news  label  \
0      WASHINGTON  —   In Sonny Perdue’s telling, Geo...      0   
1      HOUSTON  —   Venezuela had a plan. It was a ta...      0   
2      Sunday on ABC’s “This Week,” while discussing ...      0   
3      AUGUSTA, Me.  —   The beleaguered Republican g...      0   
4        Finian Cunningham has written extensively on...      1   
...                                                  ...    ...   
20795  No, you'll be a dog licking of the vomit of yo...      1   
20796  By Rixon Stewart on November 5, 2016 Rixon Ste...      1   
20797  posted by Eddie You know the Dakota Access Pip...      1   
20798  It’s officially summer, and the Society Boutiq...      0   
20799  Emory University in Atlanta, Georgia, has anno...      0   

                                           Headline_News  \
0      WASHINGTON  —   In Sonny Perdue’s telling, Geo...   
1      HOUSTON  —   Venezuela had a plan. It was a ta...   
2      Sunday on ABC’s “This Week,” while discussing ...   
3      AUGUSTA, Me.  —   The beleaguered Republican g...   
4        Finian Cunningham has written extensively on...   
...                                                  ...   
20795  No, you'll be a dog licking of the vomit of yo...   
20796  By Rixon Stewart on November 5, 2016 Rixon Ste...   
20797  posted by Eddie You know the Dakota Access Pip...   
20798  It’s officially summer, and the Society Boutiq...   
20799  Emory University in Atlanta, Georgia, has anno...   

                                   Headline_Written_News  
0      Ethics Questions Dogged Agriculture Nominee as...  
1      U.S. Must Dig Deep to Stop Argentina’s Lionel ...  
2      Cotton to House: ’Do Not Walk the Plank and Vo...  
3      Paul LePage, Besieged Maine Governor, Sends Co...  
4      A Digital 9/11 If Trump Wins Finian Cunningham...  
...                                                  ...  
20795   NeverSurrender No, you'll be a dog licking of...  
20796  Albert Pike and the European Migrant Crisis Ri...  
20797  Dakota Access Caught Infiltrating Protests to ...  
20798  How to Stretch the Summer Solstice - The New Y...  
20799  Emory University to Pay for ’100 Percent’ of U...  

[20800 rows x 7 columns]